In [7]:
import pandas as pd
melb_data = pd.read_csv('data/melb_data_ps.csv', sep = ',')
melb_data.head(3)


,index,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,...,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount,Сoordinates
0,0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,...,1,202.0,126.0,1970,Yarra,-37.7996,144.9984,Northern Metropolitan,4019,"-37.7996, 144.9984"
1,1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,...,0,156.0,79.0,1900,Yarra,-37.8079,144.9934,Northern Metropolitan,4019,"-37.8079, 144.9934"
2,2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,...,0,134.0,150.0,1900,Yarra,-37.8093,144.9944,Northern Metropolitan,4019,"-37.8093, 144.9944"


In [8]:
melb_df = melb_data.copy()

## Мы можем написать некоторую функцию, которая принимает на вход один элемент столбца, каким-то образом его обрабатывает и возвращает результат, после чего применить эту функцию к каждому элементу в столбце с помощью специального метода apply(). В результате применения этой функции будет возвращён объект Series, элементы которого будут представлять результат работы этой функции.

In [6]:
print(melb_df['Address'].nunique())

13378


In [9]:
print(melb_df['Address'].loc[177])
print(melb_df['Address'].loc[1812])
print(melb_df['Address'].loc[9001])

2/119 Railway St N
9/400 Dandenong Rd
172 Danks St


In [10]:
# На вход данной функции поступает строка с адресом.
def get_street_type(address):
# Создаём список географических пометок exclude_list.
    exclude_list = ['N', 'S', 'W', 'E']
# Метод split() разбивает строку на слова по пробелу.
# В результате получаем список слов в строке и заносим его в переменную address_list.
    address_list = address.split(' ')
# Обрезаем список, оставляя в нём только последний элемент,
# потенциальный подтип улицы, и заносим в переменную street_type.
    street_type = address_list[-1]
# Делаем проверку на то, что полученный подтип является географической пометкой.
# Для этого проверяем его на наличие в списке exclude_list.
    if street_type in exclude_list:
# Если переменная street_type является географической пометкой,
# переопределяем её на второй элемент с конца списка address_list.
        street_type = address_list[-2]
# Возвращаем переменную street_type, в которой хранится подтип улицы.
    return street_type

In [11]:
# Теперь применим эту функцию к столбцу c адресом. Для этого передадим функцию get_street_type в аргумент метода столбца apply(). В результате получим объект Series, который положим в переменную street_types:

street_types = melb_df['Address'].apply(get_street_type)
display(street_types)

0        St
1        St
2        St
3        La
4        St
         ..
13575    Cr
13576    Dr
13577    St
13578    St
13579    St
Name: Address, Length: 13580, dtype: object

In [12]:
print(street_types.nunique())

56


In [13]:
#У нас есть 56 уникальных значений. Однако наш результат можно улучшить. Давайте для начала посмотрим на частоту каждого подтипа улицы с помощью метода value_counts:
display(street_types.value_counts())

St           8012
Rd           2825
Ct            612
Dr            447
Av            321
Gr            311
Pde           211
Pl            169
Cr            152
Cl            100
La             67
Bvd            53
Tce            47
Wy             40
Avenue         40
Cct            25
Hwy            24
Parade         15
Boulevard      13
Sq             11
Crescent        9
Cir             7
Strand          7
Esplanade       6
Grove           5
Gdns            4
Grn             4
Fairway         4
Mews            4
Crossway        3
Righi           3
Victoria        2
Ridge           2
Crofts          2
Esp             2
Glade           1
Gra             1
Ave             1
Woodland        1
Outlook         1
Hts             1
Highway         1
Athol           1
Summit          1
Grand           1
Res             1
Nook            1
Eyrie           1
Dell            1
East            1
Loop            1
Grange          1
Terrace         1
Cove            1
Qy              1
Corso     

In [16]:
#  распространённый метод уменьшения количества уникальных категорий — выделим n подтипов, которые встречаются чаще всего, а остальные обозначим как 'other' (другие).
# Для этого к результату метода value_counts применим метод nlargest(), который возвращает n наибольших значений из Series. Зададим n=10, т. е. мы хотим отобрать десять наиболее популярных подтипов. Извлечём их названия с помощью атрибута index, а результат занесём в переменную popular_stypes:
popular_stypes =street_types.value_counts().nlargest(10).index
print(popular_stypes)

Index(['St', 'Rd', 'Ct', 'Dr', 'Av', 'Gr', 'Pde', 'Pl', 'Cr', 'Cl'], dtype='object')


In [17]:
# еперь, когда у нас есть список наиболее популярных подтипов улиц, введём lambda-функцию, которая будет проверять, есть ли строка x в этом перечне, и, если это так, lambda-функция будет возвращать x, в противном случае она будет возвращать строку 'other'. Наконец, применим такую функцию к Series street_types, полученной ранее, а результат определим в новый столбец таблицы StreetType:
melb_df['StreetType'] = street_types.apply(lambda x: x if x in popular_stypes else 'other')
display(melb_df['StreetType'])

0           St
1           St
2           St
3        other
4           St
         ...  
13575       Cr
13576       Dr
13577       St
13578       St
13579       St
Name: StreetType, Length: 13580, dtype: object

In [18]:
print(melb_df['StreetType'].nunique())

11


In [19]:
# Теперь, у нас нет потребности хранить признак Address, так как, если конкретное местоположение объекта всё же и влияет на его стоимость, то оно определяется столбцами Longitude и Lattitude. Удалим его из нашей таблицы:
melb_df = melb_df.drop('Address', axis =1)

In [25]:
melb_df['Date'] = pd.to_datetime(melb_df['Date'])
melb_df['WeekdaySale'] = melb_df['Date'].dt.dayofweek
display(melb_df.head(3))

,index,Suburb,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount,Сoordinates,StreetType,WeekdaySale
0,0,Abbotsford,2,h,1480000.0,S,Biggin,2016-03-12,2.5,3067,...,126.0,1970,Yarra,-37.7996,144.9984,Northern Metropolitan,4019,"-37.7996, 144.9984",St,5
1,1,Abbotsford,2,h,1035000.0,S,Biggin,2016-04-02,2.5,3067,...,79.0,1900,Yarra,-37.8079,144.9934,Northern Metropolitan,4019,"-37.8079, 144.9934",St,5
2,2,Abbotsford,3,h,1465000.0,SP,Biggin,2017-04-03,2.5,3067,...,150.0,1900,Yarra,-37.8093,144.9944,Northern Metropolitan,4019,"-37.8093, 144.9944",St,0


In [29]:
melb_df['DayType'] = melb_df['WeekdaySale'].apply(lambda x: 1 if x == (5 or 6) else 0)
display(melb_df['DayType'].value_counts())


1    7896
0    5684
Name: DayType, dtype: int64

In [31]:
average_price_w = sum(melb_df['DayType']*melb_df['Price']) / sum(melb_df['DayType'])
print(average_price_w)

1077158.690982776


In [34]:
# 4.2
def get_weekend(weekday):
    if weekday == 5 or weekday == 6:
        return 1
    else: 
        return 0
melb_df['Weekend'] = melb_df['WeekdaySale'].apply(get_weekend)
print(round(melb_df[melb_df['Weekend']==1]['Price'].mean(), 2))

1083291.7


In [33]:
display(melb_df.head(4))

,index,Suburb,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount,Сoordinates,StreetType,WeekdaySale,DayType,Weekend
0,0,Abbotsford,2,h,1480000.0,S,Biggin,2016-03-12,2.5,3067,...,Yarra,-37.7996,144.9984,Northern Metropolitan,4019,"-37.7996, 144.9984",St,5,1,1
1,1,Abbotsford,2,h,1035000.0,S,Biggin,2016-04-02,2.5,3067,...,Yarra,-37.8079,144.9934,Northern Metropolitan,4019,"-37.8079, 144.9934",St,5,1,1
2,2,Abbotsford,3,h,1465000.0,SP,Biggin,2017-04-03,2.5,3067,...,Yarra,-37.8093,144.9944,Northern Metropolitan,4019,"-37.8093, 144.9944",St,0,0,0
3,3,Abbotsford,3,h,850000.0,PI,Biggin,2017-04-03,2.5,3067,...,Yarra,-37.7969,144.9969,Northern Metropolitan,4019,"-37.7969, 144.9969",other,0,0,0


In [36]:
# 4.3
popular_realtors =melb_df['SellerG'].value_counts().nlargest(10).index
display(popular_realtors)

Index(['Nelson', 'Jellis', 'hockingstuart', 'Barry', 'Ray', 'Marshall',
       'Buxton', 'Biggin', 'Brad', 'Fletchers'],
      dtype='object')

In [39]:
melb_df['SellerG_mod'] = melb_df['SellerG'].apply(lambda x: x if x in popular_realtors else 'other')
display(melb_df['SellerG_mod'].value_counts())

other            5493
Nelson           1565
Jellis           1316
hockingstuart    1167
Barry            1011
Ray               701
Marshall          659
Buxton            632
Biggin            393
Brad              342
Fletchers         301
Name: SellerG_mod, dtype: int64

In [46]:
b = melb_df[melb_df['SellerG_mod']== 'other']['Price'].min()
a = melb_df[melb_df['SellerG_mod']== 'Nelson']['Price'].min()
print(round(a/b,1))

2.0


In [47]:
# 4.3 SOLUTION
popular_seler = melb_df['SellerG'].value_counts().nlargest(49).index
melb_df['SellerG'] = melb_df['SellerG'].apply(lambda x: x if x in popular_seler else 'other')

  
b = melb_df[melb_df['SellerG'] == 'other']['Price'].min()
print(round(a/b, 1))

1.3


In [ ]:
def get_experience(arg):
    month_key_words = ['месяц', 'месяцев', 'месяца']
    year_key_words = ['год', 'лет', 'года']
    args_splited = arg.split(' ')
    month = 0
    year = 0
    for i in range(len(args_splited))
        if args_splited[i] in month_key_words:
            month = args_splited[i-1]
        if args_splited[i] in year_key_words:
            year = args_splited[i-1]
    return int(year)*12 + int(month)
    